# Глава 22. Основы программирования модулей

## Создание модуля

Любой файл с расширением `.py` и кодом на Python автоматически будет считаться **модулем** Python.

Все имена, которым будет выполнено присваивание на верхнем уровне модуля, станут его **атрибутами**.

Для главных файлов программ, которые будут запускаться, но не будут импортироваться, имена не обязательно должны иметь расширение `.py`, однако было бы желательно использовать это расширеное, потому что оно делает назначение файлов более очевидным и позволит в будущем импортировать любой из ваших файлов.

Поскольку **имена модулей** внутри программы превращаются в имена переменных (без расширения), они также **должны следовать правилам именования обычных переменных**.

## Использование модулей

Клиенты могут использовать файл модуля, выполнив инструкцию:

* `import` - загружает модуль целиком


* `from` - загружает из модуля отдельные имена

Обе инструкции отыскивают, компилируют и запускают программный код.

### Инструкция `from`

```
# file module1.py

def printer(x):
    print(x)
```

In [1]:
# для запуска примеров импорта в этой главе
import sys
sys.path.append('./exercises/5_22/')

In [2]:
from module1 import printer

При использовании `from` модуль все равно выполняется

In [3]:
printer('text')

text


**`from <module> import *`** - импортирует все имена из `<module>`

### Импорт выполняется только один раз

**Модули загружаются и запускаются первой, и только первой инструкцией `import` или `from`**.

Реализовано такое поведение преднамеренно, потому что импортирование – это дорогостоящая операция и интерпретатор выполняет ее всего один раз за все время работы. Последующие операции импорта просто получают объект уже загруженного модуля.

Так как программный код на верхнем уровне модуля выполняется всего один раз, это обстоятельство можно использовать для инициализации переменных. Рассмотрим пример модуля `simple.py`:
```
# file simple.py

print('hello')
spam = 1 # Инициализировать переменную
```

В этом примере инструкции `print` и `=` выполняются, когда модуль импортируется впервые, и переменная `spam` инициализируется во время импортирования:

In [4]:
import simple

hello


In [5]:
simple.spam

1

Вторая и все последующие операции импортирования не приводят к перезапуску программного кода модуля – они просто получают объект модуля из внутренней таблицы модулей интерпретатора. В результате повторная инициализация переменной `spam` не происходит:

In [6]:
simple.spam = 2 # Изменить атрибут модуля

import simple # Просто получает уже загруженный модуль
simple.spam # Код не перезапускается: атрибут не изменился

2

### Инструкции `import` и `from` – операции присваивания

Импортируемые модули и имена в них не будут доступны, пока не будут выполнены соответствующие инструкции `import` или `from`. Кроме того, подобно инструкции `def`, `import` и `from` – это явные операции присваивания:

* Инструкция `import` присваивает объект модуля единственному имени


* Инструкция `from` присваивает одно или более имен объектам с теми же именами в другом модуле

```
# file small.py

x = 1
y = [1, 2]
```

In [7]:
from small import x, y  # скопировать два имени

In [8]:
x = 42  # изменяется только локальная переменная x
y[0] = 42  # изменяется непосредственно изменяемый объект

Здесь `x` не является разделяемым изменяемым объектом, а  вот `y`  – является.

Имена `y` в импортирующем и импортируемом модулях ссылаются на один и тот же объект списка, поэтому изменения, произведенные в  одном модуле, будут видны в другом модуле:

In [9]:
import small  # получить имя модуля (инстр-ия from его не дает)
small.x  # x в модуле small - это не моя переменная x

1

In [10]:
small.y # Но изменяемый объект используется совместно

[42, 2]

### Изменение значений имен в других файлах

В предыдущем примере присваивание переменной `x` в интерактивной оболочке изменяло ее значение только в этой области видимости и не оказывало влияния на переменную `x` в файле – между именем, скопированным инструкцией `from`, и именем в файле, откуда это имя было скопировано, нет никакой связи.

Чтобы действительно изменить глобальное имя в другом файле, необходимо использовать инструкцию `import`

### Эквивалентность инструкций `import` и `from`

Инструкция

```
from module import name1, name2
```

эквивалентна (но не затирает имя `module`, если оно есть до импорта)

```
import module         # Получить объект модуля
name1 = module.name1  # Скопировать имена с помощью присваивания
name2 = module.name2
del module            # Удалить имя модуля
```

> На первом шаге инструкция `from` выполняет обычную операцию `import`. Вследствие этого **инструкция `from` всегда импортирует весь модуль целиком, если он еще не был импортирован, независимо от того, сколько имен копируется из файла**.
>
>Нет никакой возможности загрузить только часть модуля (например, только одну функцию), но так как модули – это байткод, а не машинный код, влияние на производительность оказывается незначительным.

### Потенциальные проблемы инструкции `from`

Суть проблемы состоит в том, что инструкция `from` способна повреждать пространства имен, по крайней мере, в принципе – если использовать ее для импортирования переменных, когда существуют одноименные переменные в имеющейся области видимости, то эти переменные просто будут перезаписаны.

Инструкция `from` скрывает в  себе более серьезные проблемы, когда используется в комбинации с функцией `reload`, так как импортированные имена могут ссылаться на предыдущие версии объектов.

Кроме того, инструкция в форме `from module import *` действительно может повреждать пространства имен и затрудняет понимание имен, особенно когда она применяется более чем к одному файлу, – в этом случае нет никакого способа определить, какому модулю принадлежит то или иное имя, разве только выполнить поиск по файлам с исходными текстами.

В действительности форма `from *` вставляет одно пространство имен в другое, что сводит на нет преимущества, которые несет возможность разделения пространств имен.

### Когда необходимо использовать инструкцию `import`

Единственное, когда необходимо вместо инструкции `from` использовать инструкцию `import`, – когда требуется использовать одно и то же имя, присутствующее в двух разных модулях. Например, когда два файла по-разному определяют одно и то же имя и необходимо использовать обе версии имени в программе.

## Пространства имен модулей

Модули – это всего лишь пространства имен (места, где создаются имена), и имена, находящиеся в модуле, называются его атрибутами.

### Файлы создают пространства имен

Каждое имя, которому присваивается некоторое значение на верхнем уровне файла модуля (т.е. не вложенное в функции или в классы), превращается в атрибут этого модуля.

* **Инструкции модуля выполняются во время первой попытки импорта**. Когда модуль импортируется в первый раз, интерпретатор создает пустой объект модуля и выполняет инструкции в модуле одну за другой, от начала файла до конца.


* **Операции присваивания, выполняемые на верхнем уровне, создают атрибуты модуля**. Атрибуты объекта модуля сохраняются в пространстве имен модуля.


* **Доступ к пространствам имен модулей можно получить через атрибут `__dict__` или функцию `dir(M)`**. Пространства имен модулей, создаваемые операцией импортирования, представляют собой словари - доступ к ним можно получить через встроенный атрибут `__dict__`, ассоциированный с модулем, и с помощью функции `dir`. Функция `dir` - это примерный эквивалент отсортированного списка ключей атрибута `__dict__`, но она включает унаследованные имена классов, может возвращать не полный список и часто изменяется от версиии к версии.


* **Модуль - это единая область видимости (локальная является глобальной)**. В модулях *область видимости* модуля после загрузки модуля превращается в атрибут-словарь *объекта* модуля. В отличие от функций (где локальное пространство имен существует только во время выполнения функции), область видимости файла модуля превращается в область видимости атрибутов объекта модуля и никуда не исчезает после выполнения операции импортирования.

```
# файл module2.py

print('starting to load...')
import sys
name = 42
def func(): pass
class klass: pass
print('done loading.')
```

In [12]:
import module2

starting to load...
done loading.


In [13]:
module2.name

42

In [14]:
module2.sys

<module 'sys' (built-in)>

>Обратите внимание на атрибут `sys`  – инструкции `import` действительно **присваивают** объекты модулей именам, а  любая операция присваивания на верхнем уровне файла создает атрибут модуля

Внутри интерпретатора пространства имен хранятся в  виде объектов словарей. Это самые обычные объекты словарей с обычными методами. Обратиться к словарю пространства имен модуля можно через атрибут `__dict__` модуля

In [15]:
list(module2.__dict__.keys())

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__file__',
 '__cached__',
 '__builtins__',
 'sys',
 'name',
 'func',
 'klass']

Имена, которые были определены в файле модуля, становятся ключами внутреннего словаря, таким образом, большинство имен здесь отражают операции присваивания на верхнем уровне в  файле.

Однако интерпретатор Python добавляет в  пространство имен модуля еще несколько имен, например `__file__` содержит имя файла, из которого был загружен модуль, а `__name__` – это имя, под которым модуль известен импортерам (без расширения `.py` и без пути к каталогу)

### Квалификация имен атрибутов

В языке Python для доступа к атрибутам любого объекта используется **синтаксис квалификации имени `object.attribute`**.

Квалификация имени в действительности является выражением, возвращающим значение, присвоенное имени атрибута, ассоциированного с  объектом.
Например, выражение `module2.sys` в  предыдущем примере возвращает значение атрибута `sys` в объекте `module2`. Точно так же, если имеется встроенный объект списка `L`, выражение `L.append` вернет метод `append`, ассоциированный с этим списком.

* **Простые переменные**. Использование краткой формы имени, например `X`, означает, что будет произведен поиск этого имени в текущих областях видимости (следуя правилу LEGB)


* **Квалифицированные имена**. Имя `X.Y` означает, что будет произведен поиск имени `X` в текущих областях видимости, а затем будет выполнен поиск атрибута `Y` в объекте `X` (не в областях видимости)


* **Квалифицированные пути**. Имя `X.Y.Z` означает, что будет произведен поиск имени `Y` в объекте `X`, а затем поиск имени `Z` в объекте `X.Y`


* **Общий случай**. Квалификация имен применима ко всем объектам, имеющим атрибуты: модулям, классам, расширеням типов на языке C и т.д.

### Импортирование и области видимости

Невозможно получить доступ к именам, определенным в другом модуле, не импортировав его предварительно. Смысл переменной всегда определяется местоположением операции присваивания в программном коде, а для обращения к атрибутам всегда необходимо явно указывать объект.

Например, рассмотрим два следующих простых модуля. Первый, `moda.py`, определяет переменную `X`, которая является глобальной только для программного кода в этом файле, и функцию, изменяющую глобальную переменную `X` в этом файле:

```
X = 88        # Переменная X: глобальная только для этого файла
def f():      # Изменяет переменную X в этом файле
    global X
    X = 99    # Имена в других модулях недоступны
```

Второй модуль, `modb.py`, определяет свою собственную глобальную переменную `X`, а также импортирует и вызывает функцию из первого модуля:

```
X = 11        # Переменная X: глобальная только для этого файла

import moda   # Получает доступ к именам в модуле moda
moda.f()      # Изменяет переменную moda.X, но не X в этом файле

print(X, moda.X)

```

Если запустить:

In [16]:
!python ./exercises/5_22/modb.py

11 99


>Операция импортирования никогда не изменяет область видимости для программного кода в импортируемом файле - из импортируемого файла нельзя получить доступ к именам в импортирующем файле. Если быть более точным:
>* Функциям никогда не будут доступны имена, определенные  в других функциях, если только они физически не вложены друг в друга
>
>
>* Программному коду модуля никогда не будут доступны имена, определенные в других модулях, если только они явно не были импортированы

Это поведение является частью понятия **лексической области видимости**  – в языке Python области видимости, доступные части программного кода, полностью определяются физическим расположением этого программного кода в файле. Области видимости не подвержены влияниям вызовов функций или операции импортирования.

### Вложенные пространства имен

Операция импорта не дает возможности доступа к внешним областям видимости, но она дает возможность обращаться к вложенным областям видимости.

Используя квалифицированные пути к  именам атрибутов, вполне возможно
погрузиться в сколь угодно глубоко вложенные модули и получить доступ к их атрибутам.

```
# файл mod1.py
import mod2
X = 1
```

```
# файл mod2.py
import mod1
X = 2
```

In [17]:
import mod1, mod2

In [18]:
mod1.X

1

In [19]:
mod1.mod2.mod1.mod2.X

2

## Повторная загрузка модулей

Чтобы принудительно повторно загрузить модуль и запустить программный код в нем, необходимо явно вызвать **функцию `reload` модуля `imp` стандартной библиотеки**

* При вызове операции импортирования программный код модуля загружается и выполняется, только когда модуль импортируется в первый раз за время работы программы.


* При последующих попытках импортировать модуль будет использоваться объект уже загруженного модуля. Повторная загрузка и запуск программного кода в этом случае не происходит.


* Функция `reload` принудительно выполняет повторную загрузку уже загруженного модуля и запускает его программный код. Инструкции присваивания, выполняемые при повторном запуске, будут изменять существующий объект модуля.

>Функция `reload` модуля `imp` позволяет изменять части программы, не останавливая всю программу

В настоящее время функция `reload` может обслуживать только модули, написанные на языке Python

### Основы использования функции `reload`

* `reload` - это не инструкция, а функция

* функции `reload` передается существующий объект модуля, а не имя

```
import module           # Первоначальное импортирование
...исп-ся атрибуты модуля...
...                     # Теперь вып-ся изменения в файле модуля
...
from imp import reload  # Импортировать функцию reload (в 3.0)
reload(module)          # Загрузить обновленный модуль
...используются атрибуты модуля...

```

**Самое важное, что следует знать о функции `reload`**, – это то,
что она **изменяет непосредственно сам объект модуля – она не удаляет и не создает его повторно**.

Вследствие этого все ссылки на объект модуля, имеющиеся в программе, автоматически будут учитывать изменения, произошедшие в результате повторной загрузки. А  теперь подробнее о  том, как происходит повторная загрузка:

* **Функция `reload` запускает новый программный код в файле модуля в текущем пространстве имен модуля**. При повторном выполнении программный код перезаписывает существующеее пространство имен вместо того, чтобы удалять его и создавать вновь.


* **Инструкции присваивания на верхнем уровне файла замещают имена новыми значениями**.


* **Повторная загрузка оказывает воздействие на всех клиентов, использовавших инструкцию `import` для получения доступа к модулю**. Клиенты, использовавшие инструкцию `import`, получают доступ к атрибутам модуля, указывая полные их имена, поэтому после повторной загрузки они будут получать новые значения атрибутов.


* **Повторная загрузка будет воздействовать лишь на тех клиентов, которые еще только будут использовать инструкцию `from` в будущем**. Клиенты, которые использовали инструкцию `from` для получения доступа к атрибутам в прошлом, не заметят изменений, произошедших в результате повторной загрузки, - они по-прежнему будут ссылаться на старые объекты, полученные до выполнения перезагрузки.